In [ ]:
#hide
from candidate_exercises_public.core import *

# candidate-exercises-public

> This is the answer to the tech assessments for imply. Here you can find the coding exercise explained and the Case Study.

## Requirements

* python 3

## Installing

1. `git clone https://github.com/theccalderon/imply.git`
2. `cd imply`
3. `pip install -r requirements.txt`

## Execute

`python candidate_exercises_public.py path/to/json/file path/to/avro/file path/to/csv/file path/to/final/csv/file`

# Part A: Coding Excerise


The goal is to combine the files, eliminating any duplicates and write to a single .CSV file sorted alphabetically by the city name.

I decided to use `pandas` since is my go to when dealig with tabular data in python. Dataframes allow us to easily remove duplicates, aggregate data and create new files

In [ ]:
import pandas as pd
import pandavro as pdx
from pathlib import Path

In [ ]:
def combine_files(json_filepath, arvo_filepath, csv_filepath, output_filepath):
    """Combines the three files, eliminates duplicates and it sorts the resulting dataset by City Name. Creates 
    a csv file in output_filepath and returns a dataframe with its content
    """
    #reading all the three files
    df = pd.read_json(json_filepath)
    df = df.append(pd.read_csv(csv_filepath))
    df = df.append(pdx.read_avro(arvo_filepath))
    #dropping duplicates
    df = df.drop_duplicates()
    #sorting by Name
    df = df.sort_values(by='Name')
    #writing to csv
    df.to_csv(output_filepath,columns=['Name','CountryCode','Population'], index=False)
    return pd.read_csv(output_filepath)

In [ ]:
final_csv = combine_files('CityListA.json', 'CityListB.avro', 'CityList.csv', 'finalDataset.csv')

In [ ]:
!ls

00_core.ipynb              arvo.csv
99_index.ipynb             candidate_exercises_public
CONTRIBUTING.md            csv.csv
CityList.csv               docs
CityListA.json             finalDataset.csv
CityListB.avro             json.csv
LICENSE                    settings.ini
Makefile                   setup.py
README.md


`finalDataset.csv` is the resulting csv file

In [ ]:
final_csv.head()

,Name,CountryCode,Population
0,A CoruÃ±a (La CoruÃ±a),ESP,243402
1,Aalborg,DNK,161161
2,Abadan,IRN,206073
3,Abbotsford,CAN,105403
4,Aberdeen,GBR,213070


### What is the count of all rows?

To find the count of rows we can just check read the final csv file (created by `combine_files`) and check its length

In [ ]:
len(final_csv)

2083

### What is the city with the largest population?

To find the city with the largest population we can sort by Population descendingly and grab the first row

In [ ]:
def city_with_largets_population_from_df(df):
    return df.sort_values(by='Population',ascending=False)[:1][['Name']].iloc[0]['Name']

In [ ]:
print(city_with_largets_population_from_df(final_csv))

Mumbai (Bombay)


Well that's not a suprise isn't it?

### What is the total population of all cities in Brazil (CountryCode == BRA)?

To find the total population of all cities in Brazil we select the rows where CountryCode == BRA and then sum the population values.

In [ ]:
def total_population_per_country_from_df(df, country_code):
    return df.loc[df['CountryCode']==country_code]['Population'].sum()

In [ ]:
print(total_population_per_country_from_df(final_csv,'BRA'))

55955012


### What changes could be made to improve your program's performance.

The data is currently being read sequentially (one file at the time). To improve performance, we could read the data in parallel since it's coming form different sources. By doing this we would maximize the use of our resources and speed things up.

### How would you scale your solution to a much larger dataset (too large for a single machine to store)?

To scale the solution when the data cannot be stored just in a single machine we can use big data technologies (like Hadoop) which allow clustering multiple computers to analyze massive datasets in parallel more quickly. Hadoop makes it easier to use all the storage and processing capacity in cluster servers, and to execute distributed processes against huge amounts of data.

# Part B: Case Study

## From the requirements:

Dataset structure:
    
```json
{
  "timestamp": "2016-08-04T18:05:07.054Z",
  "session": "S74650219",
  "remote_address": "172.31.3.170",
  "path": "http://www.koalastothemax.com/img/koalas3.jpg",
  "referrer": "Direct",
  "timezone_offset": "-120",
  "language": "it-IT",
  "city": "Borgo San Lorenzo",
  "region": "Province of Florence",
  "country": "Italy",
  "continent": "Europe",
  "latitude": 43.9555,
  "longitude": 11.3856,
  "browser": "Mozilla",
  "browser_version": "rv:11.0",
  "agent_type": "Browser",
  "agent_category": "Personal computer",
  "os": "Windows",
  "platform": "Windows"
}
```

The site gets 200 million hits per day which means approx:
* 73 billion hits a year
* 6 billion hits a month
* 8,333,333 hits per hour
* 138,888 hits per minute

They need to answer questions mainly about unique sessions.

The on prem hardware specs is:
    - 2x 8-core HT processors (16 cores total, 32 hardware threads) 64GB memory 600GB SSD disk

## How many servers will be necessary for an analytics cluster for one year of this dataset


For this question, I'm assuming the Koala already has a deep storage system (S3, HDFS, local....) set up to store the segments. 

Since the traffic is very high we are going to use a cluster deployment. One of the main advantage of using clustering is the ability to scale horizontally on demand. That allows us to start small and then increase our hardware if needed.

For a Production setup, here is my recommendation (we need a highly available and fault tolerant setup, see https://docs.imply.io/on-prem/deploy/cluster#production-setup):
* 3 Master servers
* 2 query servers
* 4 data servers

We can start with that and monitor (using Clarity) in case we need to scale out any resources. My main concern when analyzing this question was the **data servers**, but since those can be added on the fly, we can start with 4 and increase if needed.

### Further reasearch

- roll up and metrics at ingestion time
- break apart the Data server components (see https://medium.com/airbnb-engineering/druid-airbnb-data-platform-601c312f2a4c as example) 

Sources:
* https://docs.imply.io/on-prem/deploy/cluster
* https://druid.apache.org/docs/latest/tutorials/cluster.html

## How these servers should be configured (JVM config, Druid runtime.properties)


### Comparisson between r4.2xlarge and Koala's hardware (https://aws.amazon.com/ec2/instance-types/ and https://en.wikichip.org/wiki/intel/xeon_e5/e5-2686_v4):

|             | Koala's server | AWS r4.2xlarge |
|-------------|----------------|----------------|
| Cores total | 16             | 18             |
| Threads     | 32             | 36             |
| RAM         | 64             | 61             |
| Disk        | 600 GB SSD     | Configurable   |

As per (https://docs.imply.io/on-prem/deploy/cluster#configuration-tuning)
>If you are using r4.2xlarge EC2 instances or similar hardware, the configuration in the distribution is a reasonable starting point.

Based on the table above, we can see the hardware it's actually similar.

### jvm.config

We can use the example `jvm.config` for our servers since the amount of RAM in the examples (https://docs.imply.io/on-prem/deploy/cluster) is equal to Koala's RAM per server. (see `jvm.config` files in `imply-3.1.6/dist/druid/conf/druid/cluster/`)

### runtime.properties

I'm going to list some of the values below, for more information see the `runtime.properties` files in `imply-3.1.6/dist/druid/conf/druid/cluster`:

data/middleManager/runtime.properties:druid.server.http.numThreads=60\
data/historical/runtime.properties:druid.server.http.numThreads=60\
query/broker/runtime.properties:druid.server.http.numThreads=60\
query/router/runtime.properties:druid.server.http.numThreads=100

data/historical/runtime.properties:druid.cache.sizeInBytes=256000000\
data/middleManager/runtime.properties:druid.indexer.fork.property.druid.processing.buffer.sizeBytes=100000000\
data/historical/runtime.properties:druid.processing.buffer.sizeBytes=500000000\
query/broker/runtime.properties:druid.processing.buffer.sizeBytes=500000000

.
.
.
.


## How Imply can be used to answer the two sample queries provided by Koala (number of unique sessions in a particular month; number of unique sessions per country in a particular day).

We can use `Pivot` or `SQL` to answer the questions above. Please see below how to answer the questions using `SQL`:

#### number of unique sessions in a particular month
```sql
    SELECT COUNT(DISTINCT(session)) AS unique_sessions
    FROM koala_sessions
    WHERE "__time" BETWEEN TIMESTAMP 'YYYY-MM-01 00:00:00' AND TIMESTAMP 'YYYY-(MM+1)-01 00:00:00'
```
    where koala_sessions is the datasource name and MM will be the month we want to get the information for.

#### number of unique sessions per country in a particular day
```sql
    SELECT COUNT(DISTINCT (session)) AS unique_sessions, country
    FROM koala_sessions
    WHERE "__time" BETWEEN TIMESTAMP 'YYYY-MM-DD 00:00:00' AND TIMESTAMP 'YYYY-MM-(DD+1) 00:00:00'
    GROUP BY country
```
    where koala_sessions is the datasource name and YYYY == year, MM == month, DD == day